# Import Library

In [1]:
import os
import pandas as pd
from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

# Prepocessing

In [2]:
df = pd.read_csv("news_articles.csv")
df = df.query("text_without_stopwords == text_without_stopwords")
df = df.groupby('label').apply(lambda s: s.sample(500)).reset_index(drop=True)
df.text = df["text_without_stopwords"]
df.label = pd.factorize(df.label)[0]
df = df[["text", "label"]]
df.to_csv("data/news_articles.csv",index=False)

# Set Variabel

In [3]:
PIPELINE_NAME = "gesang_wibawono-pipeline"

DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/fake_detection_transform.py"
TRAINER_MODULE_FILE = "modules/fake_detection_trainer.py"

OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

# Pipeline

In [4]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    """
    Main
    """

    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        "----direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )

In [5]:
logging.set_verbosity(logging.INFO)

from modules.components import init_components

components = init_components(
    DATA_ROOT,
    transform_module=TRANSFORM_MODULE_FILE,
    trainer_module=TRAINER_MODULE_FILE,
    training_steps=20,
    eval_steps=10,
    serving_model_dir=serving_model_dir,
)

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Pipeline root set to: output\gesang_wibawono-pipeline
INFO:absl:Generating ephemeral wheel package for 'D:\\learning\\fake-detection\\modules\\fake_detection_transform.py' (including modules: ['components', 'fake_detection_trainer', 'fake_detection_transform']).
INFO:absl:User module package has hash fingerprint version fbba7894448d71d82e81ea0710eeb6bd5c1bd6c7300c401771b1cf173c7dae84.
INFO:absl:Executing: ['C:\\Users\\BPS\\.conda\\envs\\a443-churn\\python.exe', 'C:\\Users\\BPS\\AppData\\Local\\Temp\\tmp6vwsygds\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\BPS\\AppData\\Local\\Temp\\tmpy0t2f1u9', '--dist-dir', 'C:\\Users\\BPS\\AppData\\Local\\Temp\\tmpmhliqgeg']
INFO:absl:Successfully built user code wheel distribution at 'output\\gesang_wibawono-pipeline\\_whee

INFO:absl:Node CsvExampleGen depends on [].
INFO:absl:Node CsvExampleGen is scheduled.
INFO:absl:Node Latest_blessed_model_resolver depends on [].
INFO:absl:Node Latest_blessed_model_resolver is scheduled.
INFO:absl:Node StatisticsGen depends on ['Run[CsvExampleGen]'].
INFO:absl:Node StatisticsGen is scheduled.
INFO:absl:Node SchemaGen depends on ['Run[StatisticsGen]'].
INFO:absl:Node SchemaGen is scheduled.
INFO:absl:Node ExampleValidator depends on ['Run[SchemaGen]', 'Run[StatisticsGen]'].
INFO:absl:Node ExampleValidator is scheduled.
INFO:absl:Node Transform depends on ['Run[CsvExampleGen]', 'Run[SchemaGen]'].
INFO:absl:Node Transform is scheduled.
INFO:absl:Node Trainer depends on ['Run[SchemaGen]', 'Run[Transform]'].
INFO:absl:Node Trainer is scheduled.
INFO:absl:Node Evaluator depends on ['Run[CsvExampleGen]', 'Run[Latest_blessed_model_resolver]', 'Run[Trainer]'].
INFO:absl:Node Evaluator is scheduled.
INFO:absl:Node Pusher depends on ['Run[Evaluator]', 'Run[Trainer]'].
INFO:absl

INFO:absl:Generating examples.
INFO:absl:Processing input csv data data\* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 9 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "output\\gesang_wibawono-pipeline\\CsvExampleGen\\examples\\9"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:2179316,xor_checksum:1700536015,sum_checksum:1700536015"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
, artifact_type: name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  

INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to output\gesang_wibawono-pipeline\StatisticsGen\statistics\10\Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to output\gesang_wibawono-pipeline\StatisticsGen\statistics\10\Split-eval.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 10 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'statistics': [Artifact(artifact: uri: "output\\gesang_wibawono-pipeline\\StatisticsGen\\statistics\\10"
, artifact_type: name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
base_type: STATISTICS
)]}) for execution 10
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node StatisticsGen is finished.
INFO:absl:node SchemaGen is running.
INFO:absl:Running launcher for node_info {
  ty

INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'schema': [Artifact(artifact: uri: "output\\gesang_wibawono-pipeline\\SchemaGen\\schema\\11"
, artifact_type: name: "Schema"
)]}) for execution 11
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node SchemaGen is finished.
INFO:absl:node Transform is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.transform.component.Transform"
    base_type: TRANSFORM
  }
  id: "Transform"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "gesang_wibawono-pipeline"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "20231121-100702.526989"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "gesang_wibawono-pipeline.Transform"
      }
    }
  }
}
inputs {
  inputs {
    

...

INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "TruePositives"\n        },\n        {\n          "class_name": "FalsePositives"\n        },\n        {\n          "class_name": "TrueNegatives"\n        },\n        {\n          "class_name": "FalseNegatives"\n        },\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 0.0001,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\

INFO:absl:Evaluation complete. Results written to output\gesang_wibawono-pipeline\Evaluator\evaluation\15.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to output\gesang_wibawono-pipeline\Evaluator\blessing\15.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 15 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'blessing': [Artifact(artifact: uri: "output\\gesang_wibawono-pipeline\\Evaluator\\blessing\\15"
, artifact_type: name: "ModelBlessing"
)], 'evaluation': [Artifact(artifact: uri: "output\\gesang_wibawono-pipeline\\Evaluator\\evaluation\\15"
, artifact_type: name: "ModelEvaluation"
)]}) for execution 15
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Evaluator is finished.
INFO:absl:node Pusher is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.pusher.component.Pusher"
    base_type: DEPLOY
  }
  id: "Pusher"
}
contexts {
  contexts {
    type {
      name

INFO:absl:Model version: 1700536075
INFO:absl:Model written to serving path output\serving_model\1700536075.
INFO:absl:Model pushed to output\gesang_wibawono-pipeline\Pusher\pushed_model\16.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 16 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'pushed_model': [Artifact(artifact: uri: "output\\gesang_wibawono-pipeline\\Pusher\\pushed_model\\16"
, artifact_type: name: "PushedModel"
base_type: MODEL
)]}) for execution 16
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Pusher is finished.


In [6]:
!pip freeze >> requirements.txt